# Coletando dados do twitter com Python

Esse material foi criado para servir de suporte para a palestra **Coletando dados do Twitter com Python** que foi apresentada na Python Nordeste 2022 em Aracaju/SE. 

## Conteúdo

* Slides da Apresentação
* A biblioteca Twarc
* Métodos básicos de consulta

### Slides da Apresentação 

(em breve)

### A biblioteca Twarc

Esse repositório possui scripts em python para realizar a coleta de tweets utilizando a biblioteca [TWARC](https://twarc-project.readthedocs.io/en/latest/). A biblioteca permite utilizar a versão mais nova da API do Twitter (v2), além de ter acesso aos recursos para quem tem o _Academic Access Research_. A principal escolha dessa biblioteca foi por conta da possibilidade de utilizar o acesso acadêmico, quando o usuário tem essa permissão. 

A instalação pode ser feita a partir do comando `pip install twarc` e o seu uso pode ser feito a partir da linha de comando com a chamada do comando `twarc` ou por meio de scripts em python. Esse material usa a segunda abordagem.

A biblioteca permite acesso a diferentes métodos disponibilizados pela API do twitter. Vamos trabalhar as seguintes opções: 

* Busca dos tweets recentes a partir de uma string de busca. 
* Coleta de tweets a partir do métodos de `stream` (busca em tempo real).
* Coleta de tweets postados por um usuário.



In [60]:
# imports necessários 

from twarc.expansions import flatten
from twarc.client2 import Twarc2
import jsonlines
import threading

### Acesso à API do Twitter

Para executar os scripts é necessário ter uma conta de desenvolvedor no Twitter. O cadastro de desenvolvedor pode ser feito no link: https://developer.twitter.com/. A chave utilizada vai ser o `Bearer Token` que pode ser gerado diretamente no portal de desenvolvedor. Essa chave é necessária para conectar à versão 2.0 da API do Twitter.

Alguns acessos só são permitidos para os usuários que têm o _Academic Research Access_. Mais detalhes dos requisitos para solicitar esse tipo de acesso estão disponíveis no link: https://developer.twitter.com/en/products/twitter-api/academic-research. 

### Credencias e instância da API

In [5]:
# Credenciais 

BEARER_TOKEN = ""

In [6]:
# Instanciando a biblioteca 

api_ = Twarc2(bearer_token=BEARER_TOKEN)

### Método `Search`

A busca dos tweets recentes podem ser feitas a partir do método `search_recent` que recebe os seguintes parâmetros:

In [46]:
# Retorna um conjunto de páginas contendo `max_results` por página.
# Existe um limitação a depender do tipo de conta que esteja utilizando 
# Mas há uma limitação do próprio método que limita há um período de 7 dias.

response = api_.search_recent("#python", sort_order='recency', max_results=100)

limit_pages = 2
count_pages = 1
all_tweets = []
for page in response:
    tweets = flatten(page)
    print("Tweets coletados na página %i: %i" % (count_pages, len(tweets)))
    all_tweets.extend(tweets)
    if count_pages == limit_pages:
        break 

    count_pages += 1

print("Total coletado: %i" % len(all_tweets))


Tweets coletados na página 1: 100
Tweets coletados na página 2: 100
Total coletado: 200


In [51]:
# Imprimindo algums informações dos tweets coletados
print_basic_information(all_tweets[0])

Criado em: 2022-08-24T19:53:52.000Z
Criado por: TCrossmd
Texto: RT @DrMElhosseini: ✅ Featured Playlist  | Jul - Aug Month
#python #python3 #pythonprogramming #datascience #machinelearning #dataanalysis #… 
Tweets referenciados:
Tipo: retweeted
Texto: ✅ Featured Playlist  | Jul - Aug Month
#python #python3 #pythonprogramming #datascience #machinelearning #dataanalysis #datavisualization #computerscience #computerprogramming #computerengineering #operatingsystems #computerdesign #deeplearning #ai #ml https://t.co/aJZj7cXjA7 
-------


**Salvando as informações em um arquivo:**

Ao invés de imprimir o resultado da busca na tela, vamos salvar em um arquivo `.jsonl`. Esse arquivo aceita o mesmo formato do json, com a diferença que cada linha armazena um arquivo json distinto. 

In [53]:
with jsonlines.open("../output/arquivoteste.jsonl", mode="a") as writer:
    writer.write_all(all_tweets)

## Buscando em um `timeline`

In [57]:
response_timeline = api_.timeline(user="adolfoguimaraes", exclude_retweets=True, exclude_replies=True, max_results=10)

limit_pages = 2
count_pages = 1
all_tweets_timeline = []
for page in response_timeline:
    tweets = flatten(page)
    print("Tweets coletados na página %i: %i" % (count_pages, len(tweets)))
    all_tweets_timeline.extend(tweets)
    if count_pages == limit_pages:
        break 

    count_pages += 1

print("Total coletado: %i" % len(all_tweets_timeline))


Tweets coletados na página 1: 10
Tweets coletados na página 2: 10
Total coletado: 20


**Salvando dados em um arquivo:**

In [59]:
with jsonlines.open("../output/arquivoteste2.jsonl", mode="a") as writer:
    writer.write_all(all_tweets_timeline)

## Busca em tempo real (`Stream`)

In [79]:
# Regras de busca
new_rules = [{"value": "#pythonnordeste", "tag": "pythonnordeste"}]
rules_ = api_.add_stream_rules(new_rules)

In [ ]:
# Coletando tweets
event = threading.Event()

limit_ = 5

for count, result in enumerate(api_.stream(event=event)):
    tweet_ = flatten(result)
    for t in tweet_:
        print_basic_information(t)

    if count >= limit_ - 1:
        event.set()

In [81]:
rule_ids = [r["id"] for r in rules_["data"]]
api_.delete_stream_rule_ids(rule_ids)

{'meta': {'sent': '2022-08-24T20:56:15.791Z',
  'summary': {'deleted': 1, 'not_deleted': 0}}}

In [82]:
api_.get_stream_rules()

{'meta': {'sent': '2022-08-24T20:56:19.790Z', 'result_count': 0}}

## Métodos de Apoio

In [50]:
def print_basic_information(tweet):
    print("Criado em: %s" % tweet['created_at'])
    print("Criado por: %s" % tweet['author']['username'])
    print("Texto: %s " % tweet['text'])
    if 'referenced_tweets' in tweet.keys():
        print("Tweets referenciados:")
        for ref in tweet['referenced_tweets']:
            print("Tipo: %s" % ref['type'])
            print("Texto: %s " % ref['text'])
            print("-------")

In [10]:
for pages in teste:
    print(pages)

{'data': [{'id': '1562516825254293504', 'created_at': '2022-08-24T19:06:55.000Z', 'source': 'Jovia', 'possibly_sensitive': False, 'lang': 'en', 'conversation_id': '1562516825254293504', 'entities': {'mentions': [{'start': 3, 'end': 13, 'username': 'BeingOvee', 'id': '2984376936'}, {'start': 34, 'end': 44, 'username': 'BeingOvee', 'id': '2984376936'}], 'hashtags': [{'start': 15, 'end': 18, 'tag': 'AI'}, {'start': 19, 'end': 27, 'tag': 'Chatbot'}, {'start': 46, 'end': 69, 'tag': 'ArtificialIntelligence'}, {'start': 70, 'end': 73, 'tag': 'AI'}, {'start': 74, 'end': 77, 'tag': 'ML'}, {'start': 78, 'end': 90, 'tag': 'DataScience'}, {'start': 91, 'end': 106, 'tag': 'DataScientists'}, {'start': 107, 'end': 118, 'tag': 'innovation'}, {'start': 120, 'end': 125, 'tag': 'Tech'}]}, 'context_annotations': [{'domain': {'id': '30', 'name': 'Entities [Entity Service]', 'description': 'Entity Service top level domain, every item that is in Entity Service should be in this domain'}, 'entity': {'id': '84

KeyboardInterrupt: 